## Junior Data Engineer Test _ Nucleus BI

WHINAR KUKUH RIZKY ARDANA



In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd

In [2]:
driver = webdriver.Chrome('./chromedriver')

In [3]:
link = 'https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanbod/'

In [4]:
driver.get(link)

In [725]:
# faced some strange problems that website load performs different in each loop, 
# for example sometime in my first run, there is no problem with index 10, but index 10 can be errors in my 2nd or other run
# that only problems that i faced, however the codes can get needed data even  it's in other page

# for this task, i detected 3 different of cars page category which regular/new, occasional and grouped occasional
# all of them has different kind of handling way so i divided it into 3 part of scraping

## Scraping Algorithm


#### for new or occasion car

- get all of the list of cards from first page
- loop all of the cardslist
- check if the car was new or occasion
    - take the Manufacture, Models and Price from the first page
    - get the clickable element to go to the detail page
    - click button and move to details page
        - this where the different handling between new car and occasional since it has different html structure
        - get the fuels and chassis informations
        - back to the first page
    - continue looping to other card
- if the list already in last index, then click load button to load 15 more cars
    
    
#### for grouped occasion car    
- get all of the list of cards from first page
- loop all of the cardslist
- check if the was occasion
    - get all of the list of cards from second page (there is more list in layer 2)
    - loop all of the cardslist
        - take the Manufacture, Models and Price from the first page
        - get the clickable element to go to the detail page
        - click button and move to details page
            - this where the different handling between new car and occasional since it has different html structure
            - get the fuels and chassis informations
            - back to the first page
        - continue looping to other card
    - continue looping to other card
- if the list already in last index, then click load button to load 15 more cars

## Code for Regular/New Car data

example link : `https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanbod/fiat/panda-mhev`

In [ ]:
# code untuk car regular

manufactures = []
models = []
prices = []
fuels = []
chassis = []
status = []
kilometer = []
transmisi = []

idx = 0

count_cards = len(driver.find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW'))

cur_url = driver.current_url

while idx < count_cards:
    print(idx)
    
    start_time = time.time()
#     time.sleep(5)
    element = WebDriverWait(driver, 100).until(
        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.XbA-Da"))
    )
    print('time wait: ', time.time() - start_time)
    
    card = element[0].find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW')[idx]
    occasion = card.find_elements_by_class_name('sc-jItPRW.bUgSzz')
        
    if (occasion):
        if idx == (count_cards-1):
    #         setelah back ke halaman awal tunggu hingga semua data cards mobil ter load
            element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
            )

            add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

            if (add_button) :

                driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
    #             tambah jumlah data karena tombol add sudah di klik
                time.sleep(2)
                count_cards = len(WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
                ))
                print('count cards: ',count_cards)
                
                cur_url = driver.current_url
                idx += 1
            else:
                idx += 1
                continue
        else:
            idx += 1
            continue
    else:
        names = card.find_element_by_class_name('sc-jLgLzE.eaIOqM').find_element_by_tag_name('span').text
        names = names.strip().split(' ')
    #     get manufactures
        manufactures.append(names[0])
        print('Manufacture')

        #     get models
        model = ''
        for name in names[1:]:
            model = model + f' {name}'
        model = model.lstrip()
        models.append(model)
        print('Model')

    #     get prices
        txt_price = card.find_element(By.CLASS_NAME, 'sc-Gppvi.kvInjJ ').text
        price = int(re.search(r'(\d)+', txt_price).group())
        prices.append(price)
        print('Price')
    
    
#     move page session
        link = card.find_element_by_class_name('sc-jLgLzE.eaIOqM').find_element_by_tag_name('a').get_attribute('href')
    
#     click to move page
        element = card.find_element_by_class_name('sc-fwPIEZ.iNNfbs')
        driver.execute_script("arguments[0].click();", element)

    #     wait page to complete load
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "sc-daa54801-0.dBGuIu"))
        )
        list_spec = element.find_elements(By.TAG_NAME,"li")

        fuel = (list_spec[5].text).split(':')[1].lstrip()
        fuels.append(fuel)

        body = (list_spec[1].text).split(':')[1].lstrip()
        chassis.append(body)
        
        transmisi_data = (list_spec[4].text).split(':')[1].lstrip()
        transmisi.append(transmisi_data)
        
#         
        kilometer.append(0) 
        
        status.append('new')
        
#     Back to first page
    driver.get(cur_url)
    
#     END
    if idx >= count_cards:
        break
    
    #     check is there a button for load 15 more data?
    if idx == (count_cards-1):
        
#         after move page, wait until below elements appear
        element = WebDriverWait(driver, 100).until(
                EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
        )
    
        add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

        if (add_button) :
            
            driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
#             load more data after klik button
            time.sleep(2)
            count_cards = len(WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
            ))
            print('count cards: ',count_cards)
            cur_url = driver.current_url
            idx += 1
        else:
            idx += 1
            continue
    else:
        idx += 1
        continue

In [682]:
reg_data=pd.DataFrame({
    'Manufacture' : manufactures,
    'Model' : models,
    'Price' : prices,
    'Benzine' : fuels,
    'Chassis' : chassis,
    'Status' : status
})
reg_data

# Code for Occasional Car

Example occasional details page : `https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanvragen/samenstelling-auto-en-contract?carId=R724BX&condition=occasion`

In [ ]:
# code untuk car Occasional

manufactures = []
models = []
prices = []
fuels = []
chassis = []
status = []
kilometer = []
transmisi = []

idx = 0

count_cards = len(driver.find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW'))

while idx < count_cards:
    if idx == 20:
        break
    print(idx)
    
    
    element = WebDriverWait(driver, 100).until(
        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.XbA-Da"))
    )
    
    card = element[0].find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW')[idx]
    occasion = card.find_elements_by_class_name('sc-jItPRW.bUgSzz')
        
    if (occasion):
        link = card.find_element_by_class_name('sc-jLgLzE.eaIOqM').find_element_by_tag_name('a').get_attribute('href')
        
        if 'productgroep' in link:
            if idx == (count_cards-1):
                print('productgroep index terakhir')

                time.sleep(3)
        #         after move page, wait until below elements appear
                element = WebDriverWait(driver, 10).until(
                        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
                )
                add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

                if (add_button) :

                    driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
        #             load more data after klik button
                    count_cards = len(WebDriverWait(driver, 10).until(
                        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
                    ))
                    print('count cards: ',count_cards)
                    continue
                else:
                    idx += 1
                    continue
            else:
                idx += 1
                continue
        else:
            names = card.find_element_by_class_name('sc-jLgLzE.eaIOqM').find_element_by_tag_name('span').text
            names = names.strip().split(' ')
        #     get manufactures
            manufactures.append(names[0])
            print('Manufacture')

            #     get models
            model = ''
            for name in names[1:]:
                model = model + f' {name}'
            model = model.lstrip()
            models.append(model)
            print('Model')

        #     get prices
            txt_price = card.find_element(By.CLASS_NAME, 'sc-Gppvi.kvInjJ ').text
            price = int(re.search(r'(\d)+', txt_price).group())
            prices.append(price)
            print('Price')
            
            element = card.find_element_by_class_name('sc-fwPIEZ.iNNfbs')
            driver.execute_script("arguments[0].click();", element)

            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "sc-cVzyXs.kfQQcO"))
            )

            fuel = element.find_element(By.NAME, 'brandstof-value').text
            fuels.append(fuel)

            body = element.find_element(By.NAME, 'carrosserie-value').text
            chassis.append(body)

            km_stand = float(element.find_element(By.NAME, 'kilometerstand-value').text)
            kilometer.append(km_stand) 

            transmisi_data = element.find_element(By.NAME, 'transmissie-value').text
            transmisi.append(transmisi_data)

            status.append('occasion')
    else:
        if idx == (count_cards-1):
            print('bukan occasional index terakhir')

            time.sleep(3)
    #         after move page, wait until below elements appear
            element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
            )
            print('debug non occasional')
            add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

            if (add_button) :

                driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
    #             load more data after klik button
                count_cards = len(WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
                ))
                print('count cards: ',count_cards)
                continue
            else:
                idx += 1
                continue
        else:
            idx += 1
            continue

#     KE HALAMAN 1
    driver.back() 
    print(idx)
    
    #     cek apakah masih ada tombol load 15 mobil
    if idx == (count_cards-1):
        print('occasional index terakhir')
        time.sleep(3)
#         after move page, wait until below elements appear
        element = WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
        )
    
        add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

        if (add_button) :
            
            driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
#             load more data after klik button
            count_cards = len(WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
            ))
            print('count cards: ',count_cards)
        else:
            idx += 1
            continue
    else:
        idx += 1
        continue

In [539]:
# Done Occasional

df_occasional = pd.DataFrame({
    'Manufacture' : manufactures,
    'Model' : models,
    'Price' : prices,
    'Benzine' : fuels,
    'Chassis' : chassis,
    "Status" : status
})
df_occasional.to_csv('occasional.csv', index=False)

In [690]:
# concat occasional and regular data

df_or = pd.concat([reg_data,df_occasional], axis=0).reset_index(drop=True)

df_or.to_csv('df_almost_full.csv',index=False)

# Code for Grouped Occasional car

Grouped page has 2 layers of detail pages

example layer 1 : `https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanbod/productgroep=opel_corsa_benzine` 

example layer 2 : `https://www.anwb.nl/auto/private-lease/anwb-private-lease/aanvragen/samenstelling-auto-en-contract?carId=R820GV&condition=occasion` 

In [ ]:
# code untuk car group

manufactures = []
models = []
prices = []
fuels = []
chassis = []
status = []
kilometer = []
transmisi = []

idx = 0

count_cards = len(driver.find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW'))

cur_url = driver.current_url

while idx < count_cards:
    print(idx)
    
    element = WebDriverWait(driver, 100).until(
        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.XbA-Da"))
    )
    
    card = element[0].find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW')[idx]
    
    occasion = card.find_elements_by_class_name('sc-jItPRW.bUgSzz')
        
    if (occasion):
        link = card.find_element_by_class_name('sc-jLgLzE.eaIOqM').find_element_by_tag_name('a').get_attribute('href')
        
        if 'productgroep' in link:

            add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')
            if (add_button):
                driver.execute_script("arguments[0].click();", add_button[0])
            
            element = card.find_element_by_class_name('sc-fwPIEZ.iNNfbs')
            driver.execute_script("arguments[0].click();", element)
            
            time.sleep(3)
            cards_list_layer2 = len(driver.find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW'))
            
            for index in range(cards_list_layer2):
                
                element = WebDriverWait(driver, 100).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "sc-hSpgjh.cvKRla"))
                )
                
                card_element = element.find_elements(By.CLASS_NAME, 'sc-fQZEtd.bBHUGW')
                
                names = card_element[index].find_element_by_class_name('sc-jLgLzE.eaIOqM.sc-dQuwLU.drfQAQ').find_element_by_tag_name('span').text
                names = names.strip().split(' ')
            #     get manufactures
                manufactures.append(names[0])
                print('Manufacture')

                #     get models
                model = ''
                for name in names[1:]:
                    model = model + f' {name}'
                model = model.lstrip()
                models.append(model)
                print('Model')
                
                #     get prices
                txt_price = card_element[index].find_element(By.CLASS_NAME, 'sc-Gppvi.kvInjJ ').text
                price = int(re.search(r'(\d)+', txt_price).group())
                prices.append(price)
                print('Price')
                
#                 Move halaman layer 3
                element = card_element[index].find_element_by_class_name('sc-dwsFlN.cyfelv').find_element_by_tag_name('a')
                driver.execute_script("arguments[0].click();", element)
        
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "sc-cVzyXs.kfQQcO"))
                )

                fuel = element.find_element(By.NAME, 'brandstof-value').text
                fuels.append(fuel)

                body = element.find_element(By.NAME, 'carrosserie-value').text
                chassis.append(body)

                status.append('occasional group')
                
                driver.back()
            
        else:
            if idx == (count_cards-1):
                print('bukan group index terakhir')

                time.sleep(3)
        #         after move page, wait until below elements appear
                element = WebDriverWait(driver, 10).until(
                        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
                )
                add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

                if (add_button) :

                    driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
        #             load more data after klik button
                    count_cards = len(WebDriverWait(driver, 10).until(
                        EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
                    ))
                    print('count cards: ',count_cards)
                    cur_url = driver.current_url
#                     continue
                else:
                    idx += 1
                    continue
            else:
                idx += 1
                continue
            
    else:
        if idx == (count_cards-1):
            print('bukan occasional index terakhir')

            time.sleep(3)
    #         after move page, wait until below elements appear
            element = WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
            )
            print('debug non occasional')
            add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

            if (add_button) :

                driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
    #             load more data after klik button
                count_cards = len(WebDriverWait(driver, 10).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
                ))
                print('count cards: ',count_cards)
                cur_url = driver.current_url
                continue
            else:
                idx += 1
                continue
        else:
            idx += 1
            continue

#     KE HALAMAN 1
    driver.get(cur_url) 
    
    element = WebDriverWait(driver, 100).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-hnpbor.ezOELq"))
    )
    
    #     cek apakah masih ada tombol load 15 mobil
    if idx == (count_cards-1):
        print('occasional index terakhir')
        time.sleep(3)
#         after move page, wait until below elements appear
    
        add_button = driver.find_elements(By.CLASS_NAME, 'sc-ibQAlb.cenIer.sc-fThUAz.fyGVSO')

        if (add_button) :
            
            driver.execute_script("arguments[0].click();", add_button[0]) #click tambah data          
#             load more data after klik button
            count_cards = len(WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CLASS_NAME, "sc-fQZEtd.bBHUGW"))
            ))
            cur_url = driver.current_url
            print('count cards: ',count_cards)
        else:
            idx += 1
            continue
    else:
        idx += 1
        continue

In [707]:
df_occasional_group = pd.DataFrame({
    'Manufacture' : manufactures,
    'Model' : models,
    'Price' : prices,
    'Benzine' : fuels,
    'Chassis' : chassis,
    "Status" : 'occasional group'
})
df_occasional_group

,Manufacture,Model,Price,Benzine,Chassis,Status
0,Opel,Corsa,339,Benzine,Hatchback (5-deurs),occasional group
1,Opel,Corsa,339,Benzine,Hatchback (5-deurs),occasional group
2,Hyundai,KONA Elektrisch,404,Volledig elektrisch,SUV (5-deurs),occasional group
3,Hyundai,KONA Elektrisch,434,Volledig elektrisch,SUV (5-deurs),occasional group
4,Hyundai,KONA Elektrisch,434,Volledig elektrisch,SUV (5-deurs),occasional group
5,Hyundai,KONA Elektrisch,434,Volledig elektrisch,SUV (5-deurs),occasional group
6,Hyundai,KONA Elektrisch,434,Volledig elektrisch,SUV (5-deurs),occasional group
7,Hyundai,KONA Elektrisch,449,Volledig elektrisch,SUV (5-deurs),occasional group
8,Hyundai,KONA Elektrisch,449,Volledig elektrisch,SUV (5-deurs),occasional group
9,Hyundai,KONA Elektrisch,449,Volledig elektrisch,SUV (5-deurs),occasional group


In [709]:
df_all = pd.concat([df_or,df_occasional_group], axis=0)
df_all.to_csv('data_all.csv', index=False)